# Mean-Variance Optimization — Overview

**Mean-Variance Optimization (MVO)**, developed by Harry Markowitz in 1952, is the foundation of Modern Portfolio Theory. It provides a mathematical framework for constructing portfolios that maximize expected return for a given level of risk.

---

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Expected Return** | Weighted average of individual asset returns |
| **Portfolio Variance** | Accounts for asset weights, volatilities, and correlations |
| **Efficient Frontier** | Set of optimal portfolios offering highest return per unit of risk |
| **Minimum Variance Portfolio** | Lowest-risk portfolio on the efficient frontier |
| **Tangent Portfolio** | Highest Sharpe ratio portfolio (optimal risky portfolio) |

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import minimize

np.random.seed(42)

## Portfolio Mathematics

### Portfolio Expected Return

$$E[R_p] = \sum_{i=1}^{n} w_i \cdot E[R_i] = \mathbf{w}^T \boldsymbol{\mu}$$

### Portfolio Variance

$$\sigma_p^2 = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij} = \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$

Where:
- $\mathbf{w}$ = vector of portfolio weights
- $\boldsymbol{\mu}$ = vector of expected returns
- $\boldsymbol{\Sigma}$ = covariance matrix

In [ ]:
# Define assets with expected returns and covariance matrix
assets = ['Stocks', 'Bonds', 'Real Estate', 'Commodities']
n_assets = len(assets)

# Annual expected returns (realistic estimates)
expected_returns = np.array([0.10, 0.04, 0.08, 0.05])  # 10%, 4%, 8%, 5%

# Annual volatilities
volatilities = np.array([0.18, 0.06, 0.14, 0.20])  # 18%, 6%, 14%, 20%

# Correlation matrix
correlations = np.array([
    [1.00, 0.10, 0.50, 0.20],
    [0.10, 1.00, 0.15, 0.05],
    [0.50, 0.15, 1.00, 0.25],
    [0.20, 0.05, 0.25, 1.00]
])

# Covariance matrix = diag(σ) @ corr @ diag(σ)
cov_matrix = np.outer(volatilities, volatilities) * correlations

print("Expected Returns:")
for a, r in zip(assets, expected_returns):
    print(f"  {a}: {r*100:.1f}%")

print("\nCovariance Matrix:")
print(pd.DataFrame(cov_matrix, index=assets, columns=assets).round(4))

In [ ]:
def portfolio_return(weights, returns):
    """Calculate portfolio expected return."""
    return np.dot(weights, returns)

def portfolio_volatility(weights, cov_matrix):
    """Calculate portfolio volatility."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def portfolio_sharpe(weights, returns, cov_matrix, risk_free_rate=0.02):
    """Calculate portfolio Sharpe ratio."""
    ret = portfolio_return(weights, returns)
    vol = portfolio_volatility(weights, cov_matrix)
    return (ret - risk_free_rate) / vol

def negative_sharpe(weights, returns, cov_matrix, risk_free_rate=0.02):
    """Negative Sharpe for minimization."""
    return -portfolio_sharpe(weights, returns, cov_matrix, risk_free_rate)

## The Optimization Problem

### Minimize Variance for Target Return

$$\min_{\mathbf{w}} \quad \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$

Subject to:
- $\mathbf{w}^T \boldsymbol{\mu} = \mu_{target}$ (target return)
- $\mathbf{w}^T \mathbf{1} = 1$ (weights sum to 1)
- $w_i \geq 0$ (optional: no short selling)

### Or Maximize Sharpe Ratio

$$\max_{\mathbf{w}} \quad \frac{\mathbf{w}^T \boldsymbol{\mu} - r_f}{\sqrt{\mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}}}$$

In [ ]:
# Constraints
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Weights sum to 1
]

# Bounds (long-only: 0 to 1 for each weight)
bounds = tuple((0, 1) for _ in range(n_assets))

# Initial guess: equal weights
init_weights = np.array([1/n_assets] * n_assets)

# 1. Minimum Variance Portfolio
result_min_var = minimize(
    lambda w: portfolio_volatility(w, cov_matrix),
    init_weights, method='SLSQP', bounds=bounds, constraints=constraints
)
min_var_weights = result_min_var.x

# 2. Maximum Sharpe Ratio Portfolio (Tangent Portfolio)
risk_free_rate = 0.02
result_max_sharpe = minimize(
    negative_sharpe, init_weights,
    args=(expected_returns, cov_matrix, risk_free_rate),
    method='SLSQP', bounds=bounds, constraints=constraints
)
max_sharpe_weights = result_max_sharpe.x

print("Minimum Variance Portfolio:")
for asset, weight in zip(assets, min_var_weights):
    print(f"  {asset}: {weight*100:.1f}%")
print(f"  Return: {portfolio_return(min_var_weights, expected_returns)*100:.2f}%")
print(f"  Volatility: {portfolio_volatility(min_var_weights, cov_matrix)*100:.2f}%")
print(f"  Sharpe: {portfolio_sharpe(min_var_weights, expected_returns, cov_matrix, risk_free_rate):.3f}")

print("\nMaximum Sharpe Ratio Portfolio:")
for asset, weight in zip(assets, max_sharpe_weights):
    print(f"  {asset}: {weight*100:.1f}%")
print(f"  Return: {portfolio_return(max_sharpe_weights, expected_returns)*100:.2f}%")
print(f"  Volatility: {portfolio_volatility(max_sharpe_weights, cov_matrix)*100:.2f}%")
print(f"  Sharpe: {portfolio_sharpe(max_sharpe_weights, expected_returns, cov_matrix, risk_free_rate):.3f}")

## Constructing the Efficient Frontier

The efficient frontier is the set of portfolios that offer the highest expected return for each level of risk.

In [ ]:
def efficient_frontier(returns, cov_matrix, n_points=100, allow_short=False):
    """Generate the efficient frontier."""
    n_assets = len(returns)
    
    # Target returns range
    min_ret = min(returns)
    max_ret = max(returns)
    target_returns = np.linspace(min_ret, max_ret, n_points)
    
    bounds = tuple((-1, 1) if allow_short else (0, 1) for _ in range(n_assets))
    
    frontier_vols = []
    frontier_rets = []
    frontier_weights = []
    
    for target in target_returns:
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
            {'type': 'eq', 'fun': lambda w, t=target: portfolio_return(w, returns) - t}
        ]
        
        result = minimize(
            lambda w: portfolio_volatility(w, cov_matrix),
            np.array([1/n_assets] * n_assets),
            method='SLSQP', bounds=bounds, constraints=constraints
        )
        
        if result.success:
            frontier_vols.append(portfolio_volatility(result.x, cov_matrix))
            frontier_rets.append(target)
            frontier_weights.append(result.x)
    
    return np.array(frontier_vols), np.array(frontier_rets), np.array(frontier_weights)

# Generate efficient frontier
ef_vols, ef_rets, ef_weights = efficient_frontier(expected_returns, cov_matrix)

# Generate random portfolios for comparison
n_random = 5000
random_weights = np.random.dirichlet(np.ones(n_assets), n_random)
random_rets = np.array([portfolio_return(w, expected_returns) for w in random_weights])
random_vols = np.array([portfolio_volatility(w, cov_matrix) for w in random_weights])
random_sharpes = np.array([portfolio_sharpe(w, expected_returns, cov_matrix, risk_free_rate) for w in random_weights])

In [ ]:
# Visualize the efficient frontier
fig = go.Figure()

# Random portfolios (colored by Sharpe ratio)
fig.add_trace(go.Scatter(
    x=random_vols * 100, y=random_rets * 100,
    mode='markers',
    marker=dict(size=4, color=random_sharpes, colorscale='Viridis', 
                showscale=True, colorbar=dict(title='Sharpe')),
    name='Random Portfolios',
    text=[f'Sharpe: {s:.2f}' for s in random_sharpes],
    hoverinfo='text+x+y'
))

# Efficient frontier
fig.add_trace(go.Scatter(
    x=ef_vols * 100, y=ef_rets * 100,
    mode='lines',
    line=dict(color='red', width=3),
    name='Efficient Frontier'
))

# Individual assets
fig.add_trace(go.Scatter(
    x=volatilities * 100, y=expected_returns * 100,
    mode='markers+text',
    marker=dict(size=15, color='black', symbol='diamond'),
    text=assets,
    textposition='top center',
    name='Individual Assets'
))

# Key portfolios
mv_vol = portfolio_volatility(min_var_weights, cov_matrix) * 100
mv_ret = portfolio_return(min_var_weights, expected_returns) * 100
ms_vol = portfolio_volatility(max_sharpe_weights, cov_matrix) * 100
ms_ret = portfolio_return(max_sharpe_weights, expected_returns) * 100

fig.add_trace(go.Scatter(
    x=[mv_vol], y=[mv_ret],
    mode='markers',
    marker=dict(size=20, color='blue', symbol='star'),
    name='Min Variance Portfolio'
))

fig.add_trace(go.Scatter(
    x=[ms_vol], y=[ms_ret],
    mode='markers',
    marker=dict(size=20, color='green', symbol='star'),
    name='Max Sharpe Portfolio'
))

# Capital Market Line
cml_x = np.array([0, ms_vol * 1.5])
cml_slope = (ms_ret - risk_free_rate*100) / ms_vol
cml_y = risk_free_rate*100 + cml_slope * cml_x
fig.add_trace(go.Scatter(
    x=cml_x, y=cml_y,
    mode='lines',
    line=dict(color='green', width=2, dash='dash'),
    name='Capital Market Line'
))

fig.update_layout(
    title='Mean-Variance Efficient Frontier',
    xaxis_title='Volatility (%)',
    yaxis_title='Expected Return (%)',
    template='plotly_white',
    legend=dict(x=0.02, y=0.98)
)
fig

## Portfolio Weights Along the Frontier

As we move from low risk to high risk along the efficient frontier, how do the optimal allocations change?

In [ ]:
# Visualize weight transitions
fig = go.Figure()

for i, asset in enumerate(assets):
    fig.add_trace(go.Scatter(
        x=ef_rets * 100, 
        y=ef_weights[:, i] * 100,
        mode='lines',
        stackgroup='one',
        name=asset
    ))

fig.update_layout(
    title='Portfolio Weights Across the Efficient Frontier',
    xaxis_title='Target Return (%)',
    yaxis_title='Weight (%)',
    yaxis_range=[0, 100],
    template='plotly_white'
)
fig

## Limitations of Mean-Variance Optimization

### Known Issues

| Problem | Description | Mitigation |
|---------|-------------|------------|
| **Estimation Error** | Small changes in inputs → large changes in weights | Use robust estimation, shrinkage |
| **Concentration** | Often produces extreme allocations | Add weight constraints |
| **Single Period** | Ignores multi-period dynamics | Use dynamic optimization |
| **Normality Assumption** | Ignores fat tails, skewness | Use CVaR or other risk measures |
| **Transaction Costs** | Frequent rebalancing is costly | Add turnover constraints |

### Practical Extensions

1. **Black-Litterman Model**: Combine market equilibrium with investor views
2. **Robust Optimization**: Account for parameter uncertainty
3. **Risk Parity**: Equal risk contribution from each asset
4. **Factor-Based Allocation**: Optimize over factor exposures

In [ ]:
# Demonstrate sensitivity to inputs
print("Sensitivity Analysis: How small changes in expected returns affect optimal weights")
print("="*75)

# Original max Sharpe weights
print("\nOriginal Expected Returns → Max Sharpe Weights:")
for a, r, w in zip(assets, expected_returns, max_sharpe_weights):
    print(f"  {a}: E[R]={r*100:.1f}% → Weight={w*100:.1f}%")

# Perturbed returns (just 1% change in stocks)
perturbed_returns = expected_returns.copy()
perturbed_returns[0] = 0.09  # Reduce stocks from 10% to 9%

result_perturbed = minimize(
    negative_sharpe, init_weights,
    args=(perturbed_returns, cov_matrix, risk_free_rate),
    method='SLSQP', bounds=bounds, constraints=constraints
)
perturbed_weights = result_perturbed.x

print("\nPerturbed (Stocks: 10%→9%) → Max Sharpe Weights:")
for a, r, w in zip(assets, perturbed_returns, perturbed_weights):
    print(f"  {a}: E[R]={r*100:.1f}% → Weight={w*100:.1f}%")

print("\n⚠️  A 1% change in one asset's expected return can cause significant reallocation!")

## Quick Reference

### Key Formulas

| Quantity | Formula |
|----------|--------|
| Portfolio Return | $\mu_p = \mathbf{w}^T \boldsymbol{\mu}$ |
| Portfolio Variance | $\sigma_p^2 = \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$ |
| Sharpe Ratio | $S_p = \frac{\mu_p - r_f}{\sigma_p}$ |
| Two-Asset Portfolio Vol | $\sigma_p = \sqrt{w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\rho_{12}\sigma_1\sigma_2}$ |

### Key Insights

1. **Diversification works** because portfolio variance depends on correlations
2. **Efficient frontier** is always concave (diminishing returns to risk)
3. **Capital Market Line** connects risk-free rate to tangent portfolio
4. **Two-fund theorem**: Any efficient portfolio is a combination of risk-free + tangent portfolio